# Network Deliberation ABM
## Discrete state, NK Model task
Note: This file was originally intended to create initial beliefs based on a noisy version of the global maximum (a "generated signal" task). It was modified to create random initial beliefs by setting p_error to 0.5 (an "interpreted signal" task), but the name of the experiment was not changed.

In [ ]:
# Imports
import configparser
import math
import random

import networkx as nx
import nkmodel as nk
import numpy as np
from numpy import random as nprand
import repsci
import scipy.stats as spstats
from tqdm.notebook import tqdm

import netdelib.soclearn as slearn
import netdelib.soclearn.evaluate as sleval
import netdelib.soclearn.models.generated as slgen
import netdelib.soclearn.strategy as slstrat
from netdelib.topologies.factories import *

from discrete import run_discrete

In [ ]:
# Configuration
config = configparser.ConfigParser()
config.read('discrete_nk.cfg')

N = config.getint('abm', 'N')
M = config.getint('abm', 'M')
runs = config.getint('abm', 'runs')
stages = config.getint('abm', 'stages')
steps = config.getint('abm', 'steps')

bit_count = config.getint('abm', 'bit_count')
K = config.getint('abm', 'K')
p_error = config.getfloat('abm', 'p_error')
nk_exponent = config.getfloat('abm', 'nk_exponent')

small_world_k = config.getint('networks', 'small_world_k')
small_world_a = config.getfloat('networks', 'small_world_a')
barabasi_albert_m = config.getint('networks', 'barabasi_albert_m')
er_p = config.getfloat('networks', 'erdos_renyi_p')

# Configure plotting in Jupyter
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams.update({
    'figure.figsize': (20, 20),
    'axes.spines.right': False,
    'axes.spines.left': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False})

all_networks = {
    'Complete': CompleteFactory(N, M),
    'Lattice': SmallWorldFactory(N, M, small_world_k, 0),
    'Pref. Attach.': PreferentialFactory(N, M, barabasi_albert_m),
    'Small World': SmallWorldFactory(N, M, small_world_k, small_world_a),
    'LFR': LFRFactory(N, M, 1.5, 1.5, 1/4),
    'StochasticBlock': StochasticBlockFactory(N, M),
    'Random': RandomFactory(N, M, er_p),
    'Long Path': LongPathFactory(N, M),
    'Random Group': RandomGroupFactory(N, M),
}
networks = [
    (k, all_networks[k])
    for k in config.get('networks', 'enabled').split(',')]

# Configure Strategies
individual = config.getboolean('strategies', 'individual')
individual_mode = config.getint('strategies', 'individual_mode')
individual_all_bits = config.getboolean('strategies', 'individual_all_bits')
critical = config.getboolean('strategies', 'critical')
sample = config.getint('strategies', 'sample', fallback=None)
all_learning_strategies = {
    'Best Neighbor': slstrat.best_neighbor,
    'Confident Neighbor': slstrat.confident_neighbor,
    'Conform': slstrat.conform,
    'Local Majority': slstrat.local_majority,
}
learning_strategies = [
    (k, all_learning_strategies[k])
    for k in config.get('strategies', 'enabled').split(',')
]

# Run Simulation 

In [ ]:
experiment = repsci.Experiment("discrete_generated_nk", config=config)
log = experiment.get_logger()

num_net = len(networks)
num_strat = len(learning_strategies)

score = [[0 for net in range(num_net)] for strat in range(num_strat)]
distance_score = [[0 for net in range(num_net)] for strat in range(num_strat)]
correct = [[0 for net in range(num_net)] for strat in range(num_strat)]
consensus = [[0 for net in range(num_net)] for strat in range(num_strat)]
basin = [[0 for net in range(num_net)] for strat in range(num_strat)]
initial_distance = [[0 for net in range(num_net)] for strat in range(num_strat)]
converge_time = [[0 for net in range(num_net)] for strat in range(num_strat)]

score_for_strat_net_run = [[[] for net in range(num_net)] for strat in range(num_strat)]
distance_score_for_strat_net_run = [[[] for net in range(num_net)] for strat in range(num_strat)]
correct_for_strat_net_run = [[[] for net in range(num_net)] for strat in range(num_strat)]
consensus_for_strat_net_run = [[[] for net in range(num_net)] for strat in range(num_strat)]
basin_for_strat_net_run = [[[] for net in range(num_net)] for strat in range(num_strat)]
initial_distance_for_strat_net_run = [[[] for net in range(num_net)] for strat in range(num_strat)]
converge_time_for_strat_net_run = [[[] for net in range(num_net)] for strat in range(num_strat)]
max_values = []

log.info('Running trials')
for run in tqdm(range(runs)):
    log.info('  Trial {}/{}'.format(run, runs))
    model = nk.NK(bit_count, K, nk_exponent)
    objective = model.get_value
    log.info('    Finding global max')
    max_state, max_value = model.get_global_max()
    max_values.append(max_state)
    maxima = model.get_state_maxima_map()
    log.info('    Generating initial beliefs')
    # Initial beliefs are generated once for each strategy/network combo
    initial_beliefs = slgen.initial_beliefs_noisy(N, max_state, p_error=p_error)
    
    for i, (title, learning_strategy) in enumerate(learning_strategies):
        log.info('    Strategy: {}'.format(title))
        
        for j, (title2, factory) in enumerate(networks):
            log.info('      Network: {}'.format(title2))
            
            # Record time series for scores
            run_score = []
            run_result = run_discrete(
                factory,
                learning_strategy,
                initial_beliefs,
                objective,
                N, M, stages, steps,
                individual=individual,
                individual_all_bits=individual_all_bits,
                individual_mode=individual_mode,
                critical=critical,
                sample=sample)
            
            # beliefs_at_steps maps iteration to belief dict
            beliefs_at_step = run_result.current
            scores_for_step_agent = [model.get_values(beliefs) for beliefs in beliefs_at_step]
            
            # NK Model score
            # Have to convert scores from dict values to list, otherwise np gets confused
            score_for_step = [
                np.divide(np.mean(list(scores.values())), max_value)
                for scores in scores_for_step_agent]
            score_for_strat_net_run[i][j].append(score_for_step)
            
            # Fraction correct
            correct_at_step = [
                sum([1 for belief in beliefs.values() if tuple(belief) == tuple(max_state)]) / N
                for beliefs in beliefs_at_step]
            correct_for_strat_net_run[i][j].append(correct_at_step)
            
            # Fraction having final consensus
            final_beliefs = beliefs_at_step[-1]
            # Have to cast custom return objects to standard data types
            consensus_state = tuple(spstats.mode(list(final_beliefs.values())).mode[0])
            consensus_at_step = [
                sum([1 for belief in beliefs.values() if tuple(belief) == tuple(consensus_state)]) / N
                for beliefs in beliefs_at_step]
            consensus_for_strat_net_run[i][j].append(consensus_at_step)

            # Fraction having final consensus basin
            basin_state = maxima[tuple(consensus_state)]
            basin_at_step = [
                sum([1 for belief in beliefs.values()
                     if tuple(maxima[tuple(belief)]) == tuple(basin_state)]) / N
                for beliefs in beliefs_at_step]
            basin_for_strat_net_run[i][j].append(basin_at_step)
            
            # Average distance to correct solution
            distance_score_for_step = sleval.belief_distance(beliefs_at_step, max_state)
            distance_score_for_strat_net_run[i][j].append(distance_score_for_step)
            
            # Average distance to initial beliefs
            initial_distance_for_step = sleval.belief_set_distance(beliefs_at_step, initial_beliefs)
            initial_distance_for_strat_net_run[i][j].append(initial_distance_for_step)
            
            # Calculate convergence time based on NK score
            highest_score = max(score_for_step)
            converge_threshold = 0.99 * highest_score
            t_converge = None
            for t, v in enumerate(score_for_step):
                if v >= converge_threshold:
                    t_converge = t
                    break
            converge_time_for_strat_net_run[i][j].append(t_converge)
            
log.info('All runs complete')

# Mean/std score for all runs
log.info('Calculating trial statistics')
for i, (title, learning_strategy) in enumerate(learning_strategies):
    for j, (title2, factory) in enumerate(networks):
        # Average NK model score
        mean = np.mean(score_for_strat_net_run[i][j], axis=0)
        std = np.std(score_for_strat_net_run[i][j], axis=0)
        score[i][j] = (mean, std)
        # Fraction correct
        mean = np.mean(correct_for_strat_net_run[i][j], axis=0)
        std = np.std(correct_for_strat_net_run[i][j], axis=0)
        correct[i][j] = (mean, std)
        # Fraction consensus
        mean = np.mean(consensus_for_strat_net_run[i][j], axis=0)
        std = np.std(consensus_for_strat_net_run[i][j], axis=0)
        consensus[i][j] = (mean, std)
        # Fraction consensus basin
        mean = np.mean(basin_for_strat_net_run[i][j], axis=0)
        std = np.std(basin_for_strat_net_run[i][j], axis=0)
        basin[i][j] = (mean, std)
        # Distance score
        mean = np.mean(distance_score_for_strat_net_run[i][j], axis=0)
        std = np.std(distance_score_for_strat_net_run[i][j], axis=0)
        distance_score[i][j] = (mean, std)
        # Initial distance score
        mean = np.mean(initial_distance_for_strat_net_run[i][j], axis=0)
        std = np.std(initial_distance_for_strat_net_run[i][j], axis=0)
        initial_distance[i][j] = (mean, std)
        # Converge time
        mean = np.mean(converge_time_for_strat_net_run[i][j], axis=0)
        std = np.std(converge_time_for_strat_net_run[i][j], axis=0)
        converge_time[i][j] = (mean, std)

In [ ]:
out = experiment.get_filename('score.npy')
np.save(out, score)
out = experiment.get_filename('correct.npy')
np.save(out, correct)
out = experiment.get_filename('consensus.npy')
np.save(out, consensus)
out = experiment.get_filename('basin.npy')
np.save(out, basin)
out = experiment.get_filename('distance_score.npy')
np.save(out, distance_score)
out = experiment.get_filename('initial_distance.npy')
np.save(out, initial_distance)
out = experiment.get_filename('converge_time.npy')
np.save(out, converge_time)

out = experiment.get_filename('score_run.npy')
np.save(out, score_for_strat_net_run)
out = experiment.get_filename('correct_run.npy')
np.save(out, correct_for_strat_net_run)
out = experiment.get_filename('consensus_run.npy')
np.save(out, consensus_for_strat_net_run)
out = experiment.get_filename('basin_run.npy')
np.save(out, basin_for_strat_net_run)
out = experiment.get_filename('distance_score_run.npy')
np.save(out, distance_score_for_strat_net_run)
out = experiment.get_filename('initial_distance_run.npy')
np.save(out, initial_distance_for_strat_net_run)
out = experiment.get_filename('converge_time_run.npy')
np.save(out, converge_time_for_strat_net_run)


In [ ]:
plt.figure(figsize=(16, 12))
for i, (title, learning_strategy) in enumerate(learning_strategies):
    for j, (title2, factory) in enumerate(networks):
        plt.subplot(num_strat, num_net, 1 + i * num_net + j)

        y = score[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(score[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'k-')     

        y = correct[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(correct[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'b:')
        
        y = distance_score[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(distance_score[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'g--')
        
        plt.title(f'{title} - {title2}')
        if j == 0:
            plt.ylabel('Frac. Correct')
        if i == 2:
            plt.xlabel('Time Step')
        plt.grid(True)
        plt.ylim([0.0, 1])

out = experiment.get_filename('plot.png')
if individual:
    if individual_all_bits:
        title_individual = 'all-bit'
    else:
        title_individual = 'single-bit'
    if individual_mode == slearn.MODE_ALL:
        title_individual += ' every-step individual'
    elif individual_mode == slearn.MODE_FALLBACK:
        title_individual += ' fal  lback-only individual'
    elif individual_mode == slearn.MODE_BEST:
        title_individual += ' best individual'
else:
    title_individual = 'social-only'
title = "N={}, K={}, EXP={}, S={}, {}, {}".format(
    bit_count,
    K,
    nk_exponent,
    sample,
    {True: 'critical', False: 'non-critical'}[critical],
    title_individual)
plt.suptitle(title)
plt.savefig(out, dpi=300)

In [ ]:
plt.figure(figsize=(16, 12))
for i, (title, learning_strategy) in enumerate(learning_strategies):
    for j, (title2, factory) in enumerate(networks):
        plt.subplot(num_strat, num_net, 1 + i * num_net + j)

        y = score[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(score[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'k-')     

        y = consensus[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(consensus[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'b:')
        
        plt.title(f'{title} - {title2}')
        if j == 0:
            plt.ylabel('Frac. Consensus')
        if i == 2:
            plt.xlabel('Time Step')
        plt.grid(True)
        plt.ylim([0.0, 1])

out = experiment.get_filename('plot-consensus.png')
if individual:
    if individual_all_bits:
        title_individual = 'all-bit'
    else:
        title_individual = 'single-bit'
    if individual_mode == slearn.MODE_ALL:
        title_individual += ' every-step individual'
    elif individual_mode == slearn.MODE_FALLBACK:
        title_individual += ' fal  lback-only individual'
    elif individual_mode == slearn.MODE_BEST:
        title_individual += ' best individual'
else:
    title_individual = 'social-only'
title = "N={}, K={}, EXP={}, S={}, {}, {}".format(
    bit_count,
    K,
    nk_exponent,
    sample,
    {True: 'critical', False: 'non-critical'}[critical],
    title_individual)
plt.suptitle(title)
plt.savefig(out, dpi=300)

In [ ]:
plt.figure(figsize=(16, 12))
for i, (title, learning_strategy) in enumerate(learning_strategies):
    for j, (title2, factory) in enumerate(networks):
        plt.subplot(num_strat, num_net, 1 + i * num_net + j)

        y = score[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(score[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'k-')     

        y = basin[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(basin[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'b:')
        
        plt.title(f'{title} - {title2}')
        if j == 0:
            plt.ylabel('Frac. Consensus Basin')
        if i == 2:
            plt.xlabel('Time Step')
        plt.grid(True)
        plt.ylim([0.0, 1])

out = experiment.get_filename('plot-consensus-basin.png')
if individual:
    if individual_all_bits:
        title_individual = 'all-bit'
    else:
        title_individual = 'single-bit'
    if individual_mode == slearn.MODE_ALL:
        title_individual += ' every-step individual'
    elif individual_mode == slearn.MODE_FALLBACK:
        title_individual += ' fal  lback-only individual'
    elif individual_mode == slearn.MODE_BEST:
        title_individual += ' best individual'
else:
    title_individual = 'social-only'
title = "N={}, K={}, EXP={}, S={}, {}, {}".format(
    bit_count,
    K,
    nk_exponent,
    sample,
    {True: 'critical', False: 'non-critical'}[critical],
    title_individual)
plt.suptitle(title)
plt.savefig(out, dpi=300)